In [1]:
# Load the needed package
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import cmath
from scipy.fft import fft, fftfreq, ifft
from scipy.signal import find_peaks
from scipy.integrate import solve_ivp
from decimal import *
from tqdm import tqdm

In [2]:
# Load the parameters first 
# All in SI
# 2, 0.1 30
wrf = 2 * np.pi* 10e9 # 10 GHz
wmm = 2 * np.pi* 10e9 # 10 GHz
wradial = 2 * np.pi * 1e9 # 2 GHz  
waxial = 2 * np.pi * 300e6 # 300 MHz
deff = 200e-6 # 200 micron
m = 9.10938297e-31 # 9.10938297e-31 kg 
q = 1.6e-19 # 1.6e-19 C

# Here are artical parameter
VoverU = 160 
r0 = 0.01 # 0.01 m

# Here are the tank circuit
Rp = 1e7
Q = 2000
#fres = 1000e7 convlution
fres = 0

kB = 1.380649e-23 # Boltzman constant, given by Wikipedia
Temperature = 0.4 # Temperature for the tank circuit, given by article

# Define the spatial step size and the time step size

dt = 1e-12 # time step size
T = 10e-6 # total time
N = round(T/dt)

t_eval = np.linspace(0, T, N + 1)

# Motion initialization

x0, y0, z0, vx0, vy0, vz0 = 10e-6, 10e-6, 5e-6, 0.1, 0.1, 0.1
Vec0 = x0, y0, z0, vx0, vy0, vz0

# Iteration Steps
iteration_steps = 3

peak_count = 0
progress_bar = tqdm(total=T, desc='Simulated time (s)', position=0, leave=True)

Simulated time (s):   0%|          | 0/1e-05 [00:00<?, ?it/s]

In [ ]:
def Vdc(Vec, t):
    """
    DC potential

    Params
    ---------
        Vec: 6*1 array
            the vector in the phase space, [x, y, z, vx, vy, vz]
        t: float
    Output
    ---------
        Vdc: float

    """
    #q = self.ParticleParameters['charge']
    #m = self.ParticleParameters['mass']
    #waxial = self.TrapParameters['waxial']
    x, y, z, vx, vy, vz = Vec
    return 1/4/ q * m * waxial ** 2* (2 * z ** 2 - x ** 2 - y ** 2)

# Rf potential at time t, with phase space vector Vec
def Vrf(Vec, t):
    """
    RF potential

    Params
    -------
        Vec: 6*1 array
            the vector in the phase space, [x, y, z, vx, vy, vz]
        t: float
    
    Output
    -------
        Vrf: float

    """
    #q = self.ParticleParameters['charge']
    #m = self.ParticleParameters['mass']
    #wradial = self.TrapParameters['wradial']
    #wrf = self.TrapParameters['wrf']
    x, y, z, vx, vy, vz = Vec
    return m * wradial * wrf * np.sqrt(2) / q * np.cos(wrf * t) * (x ** 2 - y ** 2) /2

def Edc(Vec, t):
    """
    Electric field by DC electrode, the position gradient of Vdc

    Params
    ---------
        Vec: 6*1 array
            the vector in the phase space, [x, y, z, vx, vy, vz]
        t: float
    
    Output
    --------
        Edc: list, size = (3)

    """
    #q = self.ParticleParameters['charge']
    #m = self.ParticleParameters['mass']
    #waxial = self.TrapParameters['waxial']
    x, y, z, vx, vy, vz = Vec
    A = 1/2/q * m * waxial ** 2
    return A * x, A * y, -2 * A * z

def Erf(Vec, t):
    """
    Electric field by RF electrode, the position gradient of Vrf

    Params
    ---------
        Vec: 6*1 array
            the vector in the phase space, [x, y, z, vx, vy, vz]
        t: float
    
    Output
    ----------
        Erf: list, size = (3)

    """
    #q = self.ParticleParameters['charge']
    #m = self.ParticleParameters['mass']
    #wradial = self.TrapParameters['wradial']
    #wrf = self.TrapParameters['wrf']
    x, y, z, vx, vy, vz = Vec
    A = m * wradial * wrf * np.cos(wrf * t) * np.sqrt(2) / q
    return A * (- x), A * y, 0.

# Calculate motional deviation according to Edc and Erf for initial Run
def DevMotion_init(t, Vec, progress_bar):
    """
    Deviation of motion, the dimension is 6, representing x, y, z, vx, vy, vz

    Params
    ------
        t: float
            Time
        Vec: 6*1 array
            the vector in the phase space, [x, y, z, vx, vy, vz]
        
    Output
    ------
        Time deviation of phase vector.
        vx:float , vy: float, vz: float, ax: float, ay: float, az: float
        
    """
    #dt = self.SimulationParameters['dt']
    #q = self.ParticleParameters['charge']
    #m = self.ParticleParameters['mass']
    x, y, z, vx, vy, vz = Vec
    #print(t, x, y, z, vx, vy, vz)

    Edc_x, Edc_y, Edc_z = Edc(Vec, t)
    Erf_x, Erf_y, Erf_z = Erf(Vec, t)
    #Ex, Ey, Ez = tuple(map(sum, zip(self.Edc(Vec, t),
    #                                self.Erf(Vec, t))))
    Ex = Edc_x + Erf_x
    Ey = Edc_y + Erf_y
    Ez = Edc_z + Erf_z
    ax = q * Ex / m
    ay = q * Ey / m
    az = q * Ez / m
    progress_bar.update(dt / 6)
    return vx, vy, vz, ax, ay, az